In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import cv2
import easyocr
import numpy as np
from tqdm import tqdm

reader = easyocr.Reader(['en'], gpu=True)

def inpaint_text(frame, reader):
    results = reader.readtext(frame)
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    for (bbox, text, prob) in results:
        top_left = tuple([int(val) for val in bbox[0]])
        bottom_right = tuple([int(val) for val in bbox[2]])

        cv2.rectangle(mask, top_left, bottom_right, 255, -1)

    inpainted_frame = cv2.inpaint(frame, mask, 3, cv2.INPAINT_TELEA)
    return inpainted_frame

def process_images(input_dir, output_dir, reader):
    for root, dirs, files in os.walk(input_dir):
        for file in tqdm(files, desc=f"Processing {root}"):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                img = cv2.imread(input_path)

                cleaned_img = inpaint_text(img, reader)

                cv2.imwrite(output_path, cleaned_img)

input_directory = '/content/drive/MyDrive/DatasetforCL_SSC'
output_directory = '/content/Cleaned_DatasetforSSC'

process_images(input_directory, output_directory, reader)

print(f"Cleaned images saved to: {output_directory}")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Processing /content/drive/MyDrive/DatasetforCL_SSC: 0it [00:00, ?it/s]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive: 0it [00:00, ?it/s]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_74: 100%|██████████| 2/2 [00:04<00:00,  2.45s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_77: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_72: 100%|██████████| 2/2 [00:02<00:00,  1.19s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_75: 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_80: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_76: 100%|██████████| 2/2 [00:02<00:00,  1.13s/it]
Processing /content/drive/MyDrive/DatasetforCL_SSC/Positive/Pair_73: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]
Processing /content/drive/MyDrive/Datasetfor

Cleaned images saved to: /content/Cleaned_DatasetforSSC


In [4]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np

In [5]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [6]:
class PairedImageDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.pairs = []
        self.labels = []
        self.transform = transform

        for label_name in ['Positive', 'Negative']:
            label = 1 if label_name == 'Positive' else 0
            label_dir = os.path.join(root_dir, label_name)
            for pair_folder in os.listdir(label_dir):
                pair_path = os.path.join(label_dir, pair_folder)
                imgs = sorted([os.path.join(pair_path, f) for f in os.listdir(pair_path) if f.endswith(('.png', '.jpg'))])
                if len(imgs) == 2:
                    self.pairs.append(imgs)
                    self.labels.append(label)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path = self.pairs[idx]
        label = self.labels[idx]
        img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


In [7]:
import torch.nn.functional as F

def combined_contrastive_classification_loss(proj1, proj2, labels, class_pred1, class_pred2, margin=1.0, lambda_clf=0.5):
    dists = F.pairwise_distance(proj1, proj2)

    contrastive_loss = (1 - labels) * dists.pow(2) + \
                       labels * torch.clamp(margin - dists, min=0).pow(2)
    contrastive_loss = contrastive_loss.mean()

    clf_targets = labels.unsqueeze(1)
    clf_preds = (class_pred1 + class_pred2) / 2
    bce_loss = F.binary_cross_entropy(clf_preds, clf_targets)

    total_loss = contrastive_loss + lambda_clf * bce_loss
    return total_loss

In [ ]:
# # class ResNet18ContrastiveWithClassifier(nn.Module):
# #     def __init__(self, projection_dim=128):
# #         super(ResNet18ContrastiveWithClassifier, self).__init__()
# #         backbone = models.resnet18(pretrained=True)
# #         self.backbone = nn.Sequential(*list(backbone.children())[:-1])  
# #         self.projection_head = nn.Linear(backbone.fc.in_features, projection_dim)
# #         self.classifier_head = nn.Linear(backbone.fc.in_features, 1)

# #     def forward(self, x):
# #         features = self.backbone(x).squeeze(-1).squeeze(-1)  
# #         projection = self.projection_head(features)
# #         classification = torch.sigmoid(self.classifier_head(features))
# #         return projection, classification

# class ResNet34ContrastiveWithClassifier(nn.Module):
#     def __init__(self, projection_dim=128):
#         super(ResNet34ContrastiveWithClassifier, self).__init__()
#         backbone = models.resnet34(pretrained=True)
#         self.backbone = nn.Sequential(*list(backbone.children())[:-1])  
#         self.projection_head = nn.Linear(backbone.fc.in_features, projection_dim)
#         self.classifier_head = nn.Linear(backbone.fc.in_features, 1)

#     def forward(self, x):
#         features = self.backbone(x).squeeze(-1).squeeze(-1)  # shape: (B, 512)
#         projection = self.projection_head(features)
#         classification = torch.sigmoid(self.classifier_head(features))
#         return projection, classification

# from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# class EfficientNetB0ContrastiveWithClassifier(nn.Module):
#     def __init__(self, projection_dim=128):
#         super(EfficientNetB0ContrastiveWithClassifier, self).__init__()
#         backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
#         self.backbone_features = backbone.features
#         self.pooling = backbone.avgpool  # Global average pooling layer

#         self.feature_dim = 1280

#         self.projection_head = nn.Linear(self.feature_dim, projection_dim)

#         self.classifier_head = nn.Linear(self.feature_dim, 1)

#     def forward(self, x):
#         features = self.backbone_features(x)
#         features = self.pooling(features)
#         features = features.flatten(start_dim=1)  

#         projection = self.projection_head(features)
#         classification = torch.sigmoid(self.classifier_head(features))

#         return projection, classification

class DenseNet121ContrastiveWithClassifier(nn.Module):
    def __init__(self, projection_dim=128):
        super(DenseNet121ContrastiveWithClassifier, self).__init__()
        densenet = models.densenet121(pretrained=True)
        self.features = densenet.features  
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.feature_dim = 1024  

        self.projection_head = nn.Linear(self.feature_dim, projection_dim)
        self.classifier_head = nn.Linear(self.feature_dim, 1)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  
        projection = self.projection_head(x)
        classification = torch.sigmoid(self.classifier_head(x))
        return projection, classification


In [ ]:
# class ResNet34Encoder(nn.Module):
#     def __init__(self):
#         super(ResNet34Encoder, self).__init__()
#         base_model = models.resnet34(pretrained=True)
#         self.backbone = nn.Sequential(*list(base_model.children())[:-1])  
#         self.embedding = nn.Linear(512, 128)  

#     def forward(self, x):
#         x = self.backbone(x).squeeze()
#         x = self.embedding(x)
#         return F.normalize(x, p=2, dim=1)



In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3), 
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [16]:
train_dataset = PairedImageDataset('/content/Cleaned_DatasetforSSC', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# model = ResNet18ContrastiveWithClassifier().cuda()
#model = ResNet34ContrastiveWithClassifier().cuda()
#model = EfficientNetB0ContrastiveWithClassifier().cuda()
model = DenseNet121ContrastiveWithClassifier().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 135MB/s]


In [ ]:
# num_epochs = 100

# model.train()

# for epoch in range(num_epochs):
#     epoch_loss = 0
#     for img1, img2, labels in tqdm(train_loader):
#         img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()

#         proj1, clf1 = model(img1)
#         proj2, clf2 = model(img2)

#         loss = combined_contrastive_classification_loss(proj1, proj2, labels, clf1, clf2)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         epoch_loss += loss.item()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# torch.save(model.state_dict(), '/content/trained_resnet34_contrastive_with_classifier_ssl.pth')

num_epochs = 100
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    for img1, img2, labels in tqdm(train_loader):
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()

        proj1, clf1 = model(img1)
        proj2, clf2 = model(img2)

        loss = combined_contrastive_classification_loss(proj1, proj2, labels, clf1, clf2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

torch.save(model.state_dict(), '/content/trained_densenet121_contrastive_with_classifier_ssl.pth')

100%|██████████| 18/18 [00:14<00:00,  1.26it/s]


Epoch 1/100, Loss: 93.7419


100%|██████████| 18/18 [00:13<00:00,  1.29it/s]


Epoch 2/100, Loss: 16.9506


100%|██████████| 18/18 [00:13<00:00,  1.30it/s]


...

Epoch 98/100, Loss: 3.0386


100%|██████████| 18/18 [00:14<00:00,  1.28it/s]


Epoch 99/100, Loss: 2.2668


100%|██████████| 18/18 [00:14<00:00,  1.28it/s]

Epoch 100/100, Loss: 2.2307
